In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.impute import KNNImputer
import datetime as dt

In [ ]:
airbnb_data = pd.read_csv('AB_NYC_2019.csv')

In [ ]:
airbnb_data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [ ]:
airbnb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [ ]:
airbnb_data.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,days_since_last_review,popularity_score,estimated_annual_revenue
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,4.889500e+04
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,2018-10-04 01:47:23.910099456,1.090910,7.143982,112.781327,295.032089,32.442297,9.905284e+03
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,2011-03-28 00:00:00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,2018-07-08 00:00:00,0.040000,1.000000,0.000000,19.000000,0.147929,0.000000e+00
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,2019-05-19 00:00:00,0.370000,1.000000,45.000000,186.000000,4.040404,2.146500e+03
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2019-06-23 00:00:00,1.580000,2.000000,227.000000,365.000000,90.719208,1.125750e+04
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,2019-07-08 00:00:00,58.500000,327.000000,365.000000,3024.000000,100.000000,1.824818e+06
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,NaN,1.597283,32.952519,131.622289,370.636506,42.295873,2.950301e+04


In [ ]:
# missing values
print(airbnb_data.isnull().sum())

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64


In [ ]:
# convert 'last_review' to datetime
airbnb_data['last_review'] = pd.to_datetime(airbnb_data['last_review'], errors='coerce')

# calculate days of last review
today = pd.to_datetime('2019-07-08')
airbnb_data['days_since_last_review'] = (today - airbnb_data['last_review']).dt.days

# fill the missing values of days_since_last_review with 365 days
airbnb_data['days_since_last_review'].fillna(365, inplace=True)

# fill the missing values of reviews_per_month with 0 where number_of_reviews is 0
airbnb_data.loc[airbnb_data['number_of_reviews'] == 0, 'reviews_per_month'] = 0

# the remaining missing of reviews_per_month
imputer = KNNImputer(n_neighbors=5)
airbnb_data[['reviews_per_month']] = imputer.fit_transform(airbnb_data[['reviews_per_month']])

<ipython-input-50-03fd2c589900>:9: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [ ]:
def categorize_price(price):
    if price <= 50:
        return 'Budget'
    elif price <= 100:
        return 'Economy'
    elif price <= 200:
        return 'Standard'
    elif price <= 500:
        return 'Premium'
    else:
        return 'Luxury'

In [ ]:
airbnb_data['price_category'] = airbnb_data['price'].apply(categorize_price)

In [ ]:
airbnb_data['popularity_score'] = (airbnb_data['number_of_reviews'] / (airbnb_data['days_since_last_review'] + 1)) * 100
airbnb_data['popularity_score'] = airbnb_data['popularity_score'].clip(upper=100)

In [ ]:
airbnb_data['estimated_annual_revenue'] = airbnb_data['price'] * (airbnb_data['availability_365'] * 0.5)

In [ ]:
airbnb_data['host_type'] = airbnb_data['calculated_host_listings_count'].apply(
    lambda x: 'Professional' if x > 10 else ('Multiple Listings' if x > 1 else 'Single Listing')
)

In [ ]:
airbnb_data[['id', 'price', 'price_category', 'days_since_last_review', 'popularity_score',
             'estimated_annual_revenue', 'host_type']].head()

,id,price,price_category,days_since_last_review,popularity_score,estimated_annual_revenue,host_type
0,2539,149,Standard,262.0,3.422053,27192.5,Multiple Listings
1,2595,225,Premium,48.0,91.836735,39937.5,Multiple Listings
2,3647,150,Standard,365.0,0.000000,27375.0,Single Listing
3,3831,89,Economy,3.0,100.000000,8633.0,Single Listing
4,5022,80,Economy,231.0,3.879310,0.0,Single Listing


## analysis questions
1. How does the price vary across different neighborhoods?
2. What factors influence a listing's popularity score?
3. How does the room type affect the estimated annual revenue?

In [ ]:
fig1 = px.histogram(airbnb_data, x='price', nbins=50, title='distribution of listing prices',
                    log_y=True, labels={'price': 'Price'})
fig1.update_layout(bargap=0.2)
fig1.show()

In [ ]:
fig2 = px.bar(airbnb_data['room_type'].value_counts().reset_index(),
              x='room_type', y='count', title='distribution of room types',
              labels={'room_type': 'room type', 'count': 'count'})
fig2.show()

In [ ]:
fig3 = px.box(airbnb_data, x='neighbourhood_group', y='price', title='price distribution by borough',
              labels={'neighbourhood_group': 'neighbourhood_group', 'price': 'price'})
fig3.show()

In [ ]:
fig4 = px.scatter(airbnb_data, x='price', y='popularity_score', color='room_type',
                  title='popularity score and price by room type',
                  labels={'price': 'price', 'popularity_score': 'popularity_score'})
fig4.show()

In [ ]:
fig5 = px.violin(airbnb_data, x='room_type', y='estimated_annual_revenue',
                 title='estimated annual revenue by room type',
                 labels={'room_type': 'room type', 'estimated_annual_revenue': 'estimated_annual_revenue'})
fig5.show()

In [ ]:
corr = airbnb_data[['price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month',
                    'calculated_host_listings_count', 'availability_365', 'popularity_score']].corr()
fig6 = ff.create_annotated_heatmap(corr.values, x=corr.columns.tolist(), y=corr.index.tolist(),
                                   colorscale='RdBu_r', showscale=True)
fig6.update_layout(title='correlation_matrix')
fig6.show()

In [ ]:
# average price by room type
avg_price_room = airbnb_data.groupby('room_type')['price'].mean().reset_index()
print(avg_price_room)

         room_type       price
0  Entire home/apt  211.794246
1     Private room   89.780973
2      Shared room   70.127586


In [ ]:

# percentage of listings by host_type
host_type_percentage = airbnb_data['host_type'].value_counts(normalize=True) * 100
print(host_type_percentage)

host_type
Single Listing       66.066060
Multiple Listings    27.865835
Professional          6.068105
Name: proportion, dtype: float64


In [ ]:

#neighbourhoods by average_popularity_score
avg_neighbourhoods = airbnb_data.groupby('neighbourhood')['popularity_score'].mean().nlargest(5)
print(avg_neighbourhoods)

neighbourhood
Huguenot        100.0
Neponsit        100.0
Richmondtown    100.0
Rossville       100.0
Silver Lake     100.0
Name: popularity_score, dtype: float64
